In [10]:
from sympy import symbols, diff, Function, Rational, sin, cos, pi, Matrix, solve, nonlinsolve
from sympy.physics.mechanics import dynamicsymbols
from sympy.utilities.lambdify import lambdify
import inspect
import numpy as np

mr, mw, g, l, r, t, tau = symbols('m_r m_w g l r t, tau')
thetar, thetaw = dynamicsymbols('theta_r theta_w')

thetardot, thetardotdot = thetar.diff(t), thetar.diff(t, t)
thetawdot, thetawdotdot = thetaw.diff(t), thetaw.diff(t, t)

In [11]:
# x and y components of velocity
# First element is x velocity (sum of x component of rod mass velocity and cart velocity)
# Second element is y velocity (y component of rod mass velocity)
pv = Matrix([l * thetardot * cos(thetar) + r * thetawdot, l * thetardot * sin(thetar)])

# Kinetic energy of system, where potential energy of mass is (1/2)*m*v^2
T = Rational(1, 2) * mr * pv.dot(pv) + Rational(1, 2) * mw * (r * thetawdot)**2

# Potential energy of system, where kinetic energy of mass is m*g*h
# where h is the height above ground/resting point
# The rod mass has maximum potential energy at the top of the pendulum (i.e. l above the pivot point)
# and minimum potential energy at the bottom of the pendulum (i.e. l below the pivot point)
# thus, it's height varies between 2*l and 0
U = mr * g * l * (1 - cos(thetar))

# Total energy
L = T - U

In [12]:
system = [
    diff(L, thetardot, t) - diff(L, thetar) + tau,
    diff(L, thetawdot, t) - diff(L, thetaw) - tau
]
eqs = nonlinsolve(system, [thetardotdot, thetawdotdot])
eqs

FiniteSet((-(g*l*m_r**2*r*sin(theta_r(t)) + g*l*m_r*m_w*r*sin(theta_r(t)) + l**2*m_r**2*r*sin(2*theta_r(t))*Derivative(theta_r(t), t)**2/2 + l*m_r*tau*cos(theta_r(t)) + m_r*r*tau + m_w*r*tau)/(l**2*m_r*r*(m_r*sin(theta_r(t))**2 + m_w)), (g*l*m_r*r*sin(2*theta_r(t))/2 + l**2*m_r*r*sin(theta_r(t))*Derivative(theta_r(t), t)**2 + l*tau + r*tau*cos(theta_r(t)))/(l*r**2*(m_r*sin(theta_r(t))**2 + m_w))))

In [13]:
f = lambdify([mr, mw, g, l, r, t, tau, thetar, thetardot, thetaw, thetawdot], eqs.args[0])
inspect.getsource(f)

'def _lambdifygenerated(m_r, m_w, g, l, r, t, tau, _Dummy_229, _Dummy_227, _Dummy_228, _Dummy_226):\n    return ((-((1/2)*_Dummy_227**2*l**2*m_r**2*r*sin(2*_Dummy_229) + g*l*m_r**2*r*sin(_Dummy_229) + g*l*m_r*m_w*r*sin(_Dummy_229) + l*m_r*tau*cos(_Dummy_229) + m_r*r*tau + m_w*r*tau)/(l**2*m_r*r*(m_r*sin(_Dummy_229)**2 + m_w)), (_Dummy_227**2*l**2*m_r*r*sin(_Dummy_229) + (1/2)*g*l*m_r*r*sin(2*_Dummy_229) + l*tau + r*tau*cos(_Dummy_229))/(l*r**2*(m_r*sin(_Dummy_229)**2 + m_w))))\n'

In [30]:
# We use the Jacobian to linearize the system at a point
jacobian_A = Matrix(eqs.args[0]).jacobian([thetar, thetaw, thetardot, thetawdot])
jacobian_A

Matrix([
[2*(g*l*m_r**2*r*sin(theta_r(t)) + g*l*m_r*m_w*r*sin(theta_r(t)) + l**2*m_r**2*r*sin(2*theta_r(t))*Derivative(theta_r(t), t)**2/2 + l*m_r*tau*cos(theta_r(t)) + m_r*r*tau + m_w*r*tau)*sin(theta_r(t))*cos(theta_r(t))/(l**2*r*(m_r*sin(theta_r(t))**2 + m_w)**2) - (g*l*m_r**2*r*cos(theta_r(t)) + g*l*m_r*m_w*r*cos(theta_r(t)) + l**2*m_r**2*r*cos(2*theta_r(t))*Derivative(theta_r(t), t)**2 - l*m_r*tau*sin(theta_r(t)))/(l**2*m_r*r*(m_r*sin(theta_r(t))**2 + m_w)), 0,      -m_r*sin(2*theta_r(t))*Derivative(theta_r(t), t)/(m_r*sin(theta_r(t))**2 + m_w), 0],
[                                                                                                   -2*m_r*(g*l*m_r*r*sin(2*theta_r(t))/2 + l**2*m_r*r*sin(theta_r(t))*Derivative(theta_r(t), t)**2 + l*tau + r*tau*cos(theta_r(t)))*sin(theta_r(t))*cos(theta_r(t))/(l*r**2*(m_r*sin(theta_r(t))**2 + m_w)**2) + (g*l*m_r*r*cos(2*theta_r(t)) + l**2*m_r*r*cos(theta_r(t))*Derivative(theta_r(t), t)**2 - r*tau*sin(theta_r(t)))/(l*r**2*(m_r*sin(thet

In [31]:
jacobian_B = Matrix(eqs.args[0]).jacobian([tau])
jacobian_B

Matrix([
[-(l*m_r*cos(theta_r(t)) + m_r*r + m_w*r)/(l**2*m_r*r*(m_r*sin(theta_r(t))**2 + m_w))],
[                     (l + r*cos(theta_r(t)))/(l*r**2*(m_r*sin(theta_r(t))**2 + m_w))]])

In [32]:
# Substitute derivatives first, otherwise it seems that we end up with an expression that includes the derivatives of constants
# for example, if we substitute (x, 5) in (d/dt)x we get (d/dt)5 which is (probably) not what we want
# ALSO make sure to use sympy's pi and not numpy's (or python's) for exact result
jacobian_A_eq = jacobian_A.subs([(thetardot, 0), (thetar, pi)])
jacobian_A_eq

Matrix([
[-(-g*l*m_r**2*r - g*l*m_r*m_w*r)/(l**2*m_r*m_w*r), 0, 0, 0],
[                                    g*m_r/(m_w*r), 0, 0, 0]])

In [33]:
jacobian_B_eq = jacobian_B.subs([(thetardot, 0), (thetar, pi)])
jacobian_B_eq

Matrix([
[-(-l*m_r + m_r*r + m_w*r)/(l**2*m_r*m_w*r)],
[                      (l - r)/(l*m_w*r**2)]])

In [34]:
jacobian_A_eq.subs([(g, 10), (r, Rational(1, 2)), (l, 3), (mr, 10), (mw, 1)])

Matrix([
[110/3, 0, 0, 0],
[  200, 0, 0, 0]])

In [35]:
jacobian_B_eq.subs([(g, 10), (r, Rational(1, 2)), (l, 3), (mr, 10), (mw, 1)])

Matrix([
[49/90],
[ 10/3]])